# Twitter Scraping


Um Störungen im Betriebsnetz festzustellen, wurden jeweils die Twitter Beiträge des offiziellen SBB Accounts gescrapt. 

In [20]:
#import libraries and modules 
from App.helper.functions import process_twitter_data, get_twitter_data, add_twitter_info
import App.helper.config as config
import pandas as pd
import os

In [21]:
# import data form csv
# data = pd.read_csv('App/Daten/data_new/combined_data.csv', parse_dates=['BETRIEBSTAG', 'AN_soll', 'AB_soll', 'AN_ist', 'AB_ist'])
data1 = pd.read_csv('./Daten/two_years/combined_data_1.csv', parse_dates=['BETRIEBSTAG', 'AN_soll', 'AB_soll', 'AN_ist', 'AB_ist'])
data2 = pd.read_csv('./Daten/two_years/combined_data_2.csv', parse_dates=['BETRIEBSTAG', 'AN_soll', 'AB_soll', 'AN_ist', 'AB_ist'])
data = pd.concat([data1, data2], ignore_index=True)
data.head()

,BETRIEBSTAG,LINIEN_ID,LINIEN_TEXT,haltestelle_an,AN_soll,AN_ist,AN_diff,haltestelle_ab,AB_soll,AB_ist,AB_diff,weekday,ab_hour,ab_minute
0,2021-06-01,1251,IC3,Zürich HB,2021-06-01 07:00:00,2021-06-01 07:00:18,0.300000,Basel SBB,2021-06-01 06:06:00,2021-06-01 06:07:27,1.450000,1,6,6
1,2021-06-01,1252,IC,Basel SBB,2021-06-01 22:48:00,2021-06-01 22:47:55,-0.083333,Zürich HB,2021-06-01 21:36:00,2021-06-01 21:37:03,1.050000,1,21,36
2,2021-06-01,1253,IC3,Zürich HB,2021-06-01 08:00:00,2021-06-01 08:02:47,2.783333,Basel SBB,2021-06-01 07:06:00,2021-06-01 07:07:18,1.300000,1,7,6
3,2021-06-01,1260,IC3,Basel SBB,2021-06-01 21:53:00,2021-06-01 21:51:42,-1.300000,Zürich HB,2021-06-01 20:59:00,2021-06-01 21:00:18,1.300000,1,20,59
4,2021-06-01,1956,IR36,Basel SBB,2021-06-01 06:50:00,2021-06-01 06:51:18,1.300000,Zürich HB,2021-06-01 05:35:00,2021-06-01 05:36:17,1.283333,1,5,35


In [22]:
# set start and end date
START_DATE = data['BETRIEBSTAG'].min().strftime('%Y-%m-%d')
END_DATE = data['BETRIEBSTAG'].max().strftime('%Y-%m-%d')

# Define twitter query
query = 'from:railinfo_sbb since:' + START_DATE + ' until:' + END_DATE
max_tweets = 15000

# lowercase all items from config.HALTESTELLEN
haltestellen = [x.lower() for x in config.HALTESTELLEN]

tweets_raw = get_twitter_data(query, max_tweets, haltestellen)

# Save twitter data to csv
tweets_raw.to_csv('App/Daten/data_new/twitter_data_raw.csv', index=False)


c:\Users\stoec\OneDrive - Universität St.Gallen\big-data-fs23\App\helper\functions.py:17: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  if any(keyword in tweet.content.lower() for keyword in haltestellen):
c:\Users\stoec\OneDrive - Universität St.Gallen\big-data-fs23\App\helper\functions.py:18: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  attributes_container.append([tweet.date, tweet.likeCount, tweet.sourceLabel, tweet.content])


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500


Stopping after 20 empty pages


In [23]:
# Process twitter data
tweets = pd.read_csv('App/Daten/data_new/twitter_data_raw.csv', parse_dates=['Date Created'])


# define all lines
linien = [x.lower() for x in config.LINIEN]

# lowercase all items from config.HALTESTELLEN
haltestellen = [x.lower() for x in config.HALTESTELLEN]

# suppress Warnings
pd.options.mode.chained_assignment = None  # default='warn'
twitter_clean = process_twitter_data(tweets, haltestellen, linien)
# reactivating Warnings
pd.options.mode.chained_assignment = 'warn'  # default='warn'

# Save twitter data to csv
twitter_clean.to_csv('App/Daten/data_new/twitter_data_clean.csv', index=False)

# # Delete twitter_data_clean.csv
# try:
#     os.remove('App/Daten/twitter_data_raw.csv')
# except:
#     pass

In [24]:
#Check values
twitter_clean['Einschr_type'].value_counts()

Einschränkungen    361
Endmeldung:        194
Aufgehoben:         63
Einschränkung       28
Unterbruch          13
Verkehr              1
Teilausfall:         1
Name: Einschr_type, dtype: int64

In [25]:
# check values
twitter_clean = twitter_clean[twitter_clean['Einschr_type'] != 'Verkehr']
twitter_clean = twitter_clean[twitter_clean['Einschr_type'] != 'Teilausfall:']
twitter_clean['Einschr_type'].value_counts()         

Einschränkungen    361
Endmeldung:        194
Aufgehoben:         63
Einschränkung       28
Unterbruch          13
Name: Einschr_type, dtype: int64

In [26]:
# check if data_disturbances already exists
if not 'disturbance_overlap' in data.columns:
    twitter_clean.drop_duplicates(inplace=True)
    data_twitter = add_twitter_info(data, twitter_clean)
    data_twitter.drop_duplicates(inplace=True)
    # Save data to csv
    data_twitter1 = data_twitter.loc[len(data_twitter)//2:]
    data_twitter2 = data_twitter.loc[:len(data_twitter)//2]
    data_twitter1.to_csv('App/Daten/data_new/combined_data_twitter1.csv', index=False)
    data_twitter2.to_csv('App/Daten/data_new/combined_data_twitter2.csv', index=False)

In [30]:
print(data_twitter['BETRIEBSTAG'].min())
print(data_twitter['BETRIEBSTAG'].max())

2021-06-01 00:00:00
2023-05-31 00:00:00


In [31]:
# get range of 'Date Created' in twitter_clean
print(twitter_clean['Date Created'].min())
print(twitter_clean['Date Created'].max())

2022-05-22 14:03:32+00:00
2023-05-30 21:51:03+00:00
